# 3.1.1 Tensor

In [2]:
import torch
a = torch.Tensor([[2,3],[4,8],[7,9]])
print('a is {}'.format(a))
print('a size is {}'.format(a.size()))

a is tensor([[2., 3.],
        [4., 8.],
        [7., 9.]])
a size is torch.Size([3, 2])


In [3]:
b = torch.LongTensor([[2,3],[4,8],[7,9]])
print('b is {}'.format(b))

b is tensor([[2, 3],
        [4, 8],
        [7, 9]])


In [5]:
c = torch.zeros((3,2))
print('zero tensor is {}'.format(c))

zero tensor is tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])


In [6]:
d = torch.randn((3,2))
print('zero tensor is {}'.format(d))

zero tensor is tensor([[-1.2078,  0.1057],
        [-0.9197,  0.0104],
        [-2.2724, -0.3868]])


## 可以像numpy一样，通过索引取得其中的元素，改变它的值，如：

In [7]:
a[2,0]

tensor(7.)

In [8]:
a[2,0] = 100
print('changed a is {}'.format(a))

changed a is tensor([[  2.,   3.],
        [  4.,   8.],
        [100.,   9.]])


## 在Tensor 与 numpy.ndarray 之间相互转换
a.numpy() ---convert to numpy, __a:tensor__    
torch.__from_numpy(a) --__a:numpy array__

In [9]:
numpyB = b.numpy()
print('conver to numpy is {}'.format(numpyB))

conver to numpy is [[2 3]
 [4 8]
 [7 9]]


In [11]:
import numpy as np
e = np.array([[2,3],[3,4]])
tensorE = torch.from_numpy(e)
print('conver to tensor is {}'.format(tensorE))

conver to tensor is tensor([[2, 3],
        [3, 4]])


## change the type of tensor to float, like a.float()

In [12]:
ftorchE = tensorE.float()
print('float torch is {}'.format(ftorchE))

float torch is tensor([[2., 3.],
        [3., 4.]])


## 通过torch.cuda.is_available()判断，电脑是否支持GPU。

支持的话，如果想__把tensor a放到GPU上__，只需a.cuda()就可以。

In [13]:
torch.cuda.is_available()

False

In [14]:
if torch.cuda.is_available():
    a_cuda = a.cuda()
    print(a_cuda)

# 3.1.2 Variable  
Variable 在numpy中不存在，提供了自动求导的功能。   
TensorFlow，神经网络在做运算时，需要先构造一个计算图谱，然后，在里面进行前向传播和反向传播。  

Variable会被放入一个计算图中，然后进行前向传播，反向传播，自动求导。


In [20]:
# Tensor and Variable 没有本质区别，
from torch.autograd import Variable
a = torch.Tensor([[2,3],[4,8]])
va = Variable(a)
va

tensor([[2., 3.],
        [4., 8.]])

## Variable 有三个属性：

1. data：取出Variable里面Tensor的值
2. grad：保存了Variable的反向传播梯度
3. grad_fn：得到Variable的操作

__requires_grad=True，表示对该变量求梯度__

In [21]:
x = Variable(torch.Tensor([1]), requires_grad=True)
w = Variable(torch.Tensor([2]), requires_grad=True)
b = Variable(torch.Tensor([3]), requires_grad=True)
x

tensor([1.], requires_grad=True)

In [22]:
# Build a computational graph.
y = w*x + b

## 1. scalar gradient：标量求导，backward函数的参数可以不写。     
y.backward() is equal to y.backward(torch.FloatTensor([1]))  

## 2. 对矩阵求导，backward函数必须有参数。   
y.backward(torch.FloatTensor([1, 0.1, 0.01]))   
得到的结果就是他们每个分量的梯度---得到的梯度就是他们原本的梯度分别 *乘以 1， 0.1， 0.01

In [23]:
# Compute gradients
y.backward()

In [24]:
print(x.grad)
print(w.grad)
print(b.grad)

tensor([2.])
tensor([1.])
tensor([1.])


In [25]:
b

tensor([3.], requires_grad=True)

In [30]:
x = torch.randn(3)
x = Variable(x, requires_grad=True)
y = x * 2
#y.backward()
#print(x.grad)
y.backward(torch.FloatTensor([1, 0.1, 0.01]))
print(x.grad)

tensor([2.0000, 0.2000, 0.0200])


In [28]:
x

tensor([-0.9622,  0.2611, -0.7511], requires_grad=True)

# 3.1.3 Dataset   
__torch.utils.data.Dataset__ is abstract class(抽象类).  
定义你的数据类可以 继承和重写这个抽象类，只需要定义 \_\_len\_\_ and \_\_getitem__ two functions.  
通过迭代的方式来取得每一个数据。

In [32]:
import pandas as pd
from torch.utils.data import Dataset
class myDataset(Dataset):
    def __init__(self, csv_file, txt_file, root_dir, other_file):
        self.csv_data = pd.read_csv(csv_file)
        with open(txt_file, 'r') as f:
            data_list = f.readlines()
        self.txt_data = data_list
        self.root_dir = root_dir
        
    def __len__(self):
        return len(self.csv_data)
    
    def __getitem__(self, idx):
        data = (self.csv_data[idx], self.txt_data[idx])
        return data

__torch.utils.data.DataLoader__ 定义一个新的迭代器   
可以取batch,shuffle，或者多线程去读取数据。

In [35]:
from torch.utils.data import DataLoader
def my_collate(batch):
    data = [item[0] for item in batch]
    target = [item[1] for item in batch]
    target = torch.LongTensor(target)
    return [data, target]

dataiter = DataLoader(myDataset, batch_size=32, shuffle=True, collate_fn=my_collate)

TypeError: object of type 'type' has no len()

In [36]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt

# a simple custom collate function, just to show the idea
def my_collate(batch):
    data = [item[0] for item in batch]
    target = [item[1] for item in batch]
    target = torch.LongTensor(target)
    return [data, target]


def show_image_batch(img_list, title=None):
    num = len(img_list)
    fig = plt.figure()
    for i in range(num):
        ax = fig.add_subplot(1, num, i+1)
        ax.imshow(img_list[i].numpy().transpose([1,2,0]))
        ax.set_title(title[i])

    plt.show()

#  do not do randomCrop to show that the custom collate_fn can handle images of different size
train_transforms = transforms.Compose([transforms.Scale(size = 224),
                                       transforms.ToTensor(),
                                       ])

# change root to valid dir in your system, see ImageFolder documentation for more info
train_dataset = datasets.ImageFolder(root="/hd1/jdhao/toyset",
                                     transform=train_transforms)

trainset = DataLoader(dataset=train_dataset,
                      batch_size=4,
                      shuffle=True,
                      collate_fn=my_collate, # use custom collate function here
                      pin_memory=True)

trainiter = iter(trainset)
imgs, labels = trainiter.next()

# print(type(imgs), type(labels))
show_image_batch(imgs, title=[train_dataset.classes[x] for x in labels])

/usr/local/anaconda3/lib/python3.7/site-packages/torchvision/transforms/transforms.py:219: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


FileNotFoundError: [Errno 2] No such file or directory: '/hd1/jdhao/toyset'